# Understanding PyTorch

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/navar 

Mounted at /content/drive
/content/drive/MyDrive/navar


Here we just go through the [Quickstart tutorial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html).

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

Importing some data:

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [4]:
batch_size = 64 

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [5]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential( # network architecture - MLP
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x) # flatten inputs 
        logits = self.linear_relu_stack(x) # apply MLP and fit/predict
        return logits # return fitted values

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


Just setting a loss function and an optimizer:

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Now we still need to define funcions to train and test the model.

> "To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call model.forward() directly!"


In [21]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [13]:
def train(dataloader, model, loss_fn, optimizer):
  # Takes classes and instances we defined above:
    size = len(dataloader.dataset)
    batch_counter = 0
    for batch, (X, y) in enumerate(dataloader):
        batch_counter += 1
        print(batch_counter)
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X) # performs predictions
        loss = loss_fn(pred, y) # compute loss

        # Backpropagation
        # Intuition: train in-sample recursively
        # HUNCH(!): for each batch, train model, compute loss
        optimizer.zero_grad() # "take first order conditions" - 1 round of GD?
        loss.backward() # update loss - in-sample (I guess)
        optimizer.step() # update the optimizer to intialize on next turn

        if batch % 100 == 0: # (?)
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Finally, just run the process to train the model. Loss should tend to decrease with each batch and each epoch.

In [14]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
1
loss: 2.308011  [    0/60000]
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
loss: 2.302609  [ 6400/60000]
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
loss: 2.302292  [12800/60000]
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
24

KeyboardInterrupt: ignored

## Convolutional 1D

In [ ]:
m = nn.Conv1d(16, 33, 3, stride=2) # 16 input channels, 33 output channels, kernel size 3
input = torch.randn(20, 16, 50)
output = m(input)
# We go from 20 matrices of dim (16 x 50) to 20 matrices of size (33 x 25)
output.shape

torch.Size([20, 33, 24])